# Connect Drone WIFI

In [1]:
# Connect Tello WIFI -> Change NAME!!!
import os
import time
connect_counter = 0
success = 1
while success != 0 and connect_counter < 5:
    os.system('nmcli device wifi rescan')
    success = os.system('nmcli device wifi connect TELLO-5A37E2')
    connect_counter += 1
    time.sleep(2)
print(success)

0


# Import Python + SAS Open Source Libraries

In [2]:
from djitellopy import Tello
import cv2
import pygame
from pygame.locals import *
import numpy as np
import time
import threading
import base64
import random

import esppy
import websocket
import json
from random import randint

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Create SAS Event Stream Processing Project

In [3]:
class Esp_project():
        def __init__(self):
            esp = esppy.ESP(hostname='http://localhost:9900')
            esp_project = esp.create_project('test', n_threads=10)
            esp_project.pubsub = 'manual'
            esp_project.add_continuous_query('contquery')

            # Window: Video Capture
            vid_capture = esp.SourceWindow(schema=('id*:int64', 'image:blob'),
            index_type='empty', insert_only=True)
            vid_capture.pubsub = True
            esp_project.windows['w_input'] = vid_capture

            # Window: Video Resize
            vid_capture_resize = esp.CalculateWindow(algorithm='ImageProcessing', 
                                                     name='resized', 
                                                     function='resize',
                                                     height=416, 
                                                     width=416, 
                                                     input_map=dict(imageInput='image'), 
                                                     output_map=dict(imageOutput='_image_'))
            vid_capture_resize.schema_string = 'id*:int64,image:blob,_image_:blob'
            esp_project.windows['w_resize'] = vid_capture_resize

            # Window: Model Reader
            model_reader = esp.ModelReaderWindow()
            esp_project.windows['w_reader'] = model_reader

            # Window: Model Request
            model_request = esp.SourceWindow(schema=('req_id*:int64', 'req_key:string', 'req_val:string'),index_type='empty', insert_only=True)
            esp_project.windows['w_request'] = model_request

            # Window: Model Score
            model_score = esp.ScoreWindow()
            model_score.pubsub = True
            model_score.add_offline_model(model_type='astore')
            def score_window_fields(number_objects):
                _field = "id*:int64,image:blob,_image_:blob,_nObjects_:double,"
                for obj in range(0,number_objects):
                    _field += "_Object" + str(obj) + "_:string,"
                    _field += "_P_Object" + str(obj) + "_:double,"
                    _field += "_Object" + str(obj) + "_x:double,"
                    _field += "_Object" + str(obj) + "_y:double,"
                    _field += "_Object" + str(obj) + "_width:double,"
                    _field += "_Object" + str(obj) + "_height:double,"
                return _field[:-1]
            model_score.schema_string = score_window_fields(20)
            esp_project.windows['w_score'] = model_score

            # Connections
            vid_capture.add_target(vid_capture_resize, role='data')
            vid_capture_resize.add_target(model_score, role='data')
            model_request.add_target(model_reader, role='request')
            model_reader.add_target(model_score, role='model')

            # Load Project time delta
            esp.load_project(esp_project)

            # Publisher: Send Model
            pub = model_request.create_publisher(blocksize=1, rate=0, pause=0, dateformat='%Y%dT%H:%M:%S.%f', opcode='insert', format='csv')
            pub.send('i,n,1,"usegpuesp","1"\n')
            pub.send('i,n,2,"ndevices","1"\n')
            pub.send('i,n,3,"action","load"\n')
            pub.send('i,n,4,"type","astore"\n')
            pub.send('i,n,5,"reference","/data/models/Tiny-Yolov2.astore"\n')
            pub.send('i,n,6,,\n')
            pub.close()

            # Publisher: Send Video
            self.pub = vid_capture.create_publisher(blocksize=1, rate=0, pause=0, opcode='insert', format='csv')
            print('ESP Project loaded')
            self.esp_project = esp_project

# Create ESP Data-Subscriber

In [4]:
class Esp_project_sub():
    def __init__(self):
        self.ws = websocket.WebSocketApp("ws://localhost:9900/SASESP/subscribers/test/contquery/w_score/?format=json&mode=streaming&pagesize=1&schema=true",
                         on_message = self.on_message,
                         on_error = self.on_error,
                         on_close = self.on_close)
        self.ws.on_open = self.on_open
        
        self.frame = None
        self.cmd = None
        self.face_track = False
        return
        
    def highlightImage(self, data):
        object_list = ['Human face', 'Person', 'Suit']
        color_palette = [
        (255,64,0), #red
        (255,191,0), #orange
        (255,255,0), #yellow
        (64,255,0), #green
        (0,255,255) #blue
        ]
        obj_colors = {}
        i = 0
        for _object in object_list:
            obj_colors[_object] = color_palette[i]
            i += 1
        cmd = []

        row = data['events'][0]['event']
        numberOfObjects = data['events'][0]['event']['_nObjects_']
        imageBufferBase64 = data['events'][0]['event']['image']['image']

        nparr = np.frombuffer(base64.b64decode(imageBufferBase64), dtype=np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        image_h, image_w,_ = frame.shape
        for i in range(0, int(float(numberOfObjects))):
            obj = row['_Object' + str(i) + '_']
            prob = float(row['_P_Object' + str(i) + '_'])
            probability = " (" + str(round(prob * 100, 2)) + "%)"
            x = float(row['_Object' + str(i) + '_x'])
            y = float(row['_Object' + str(i) + '_y'])
            width = float(row['_Object' + str(i) + '_width'])
            height = float(row['_Object' + str(i) + '_height'])

            x1 = int(image_w * (x - width / 2))
            y1 = int(image_h * (y - height/ 2))
            x2 = int(image_w * (x + width / 2))
            y2 = int(image_h * (y + height/ 2))
            if self.detect_objects == 'all':
                bbox_color = (64,255,0)
                border_offset = 3
                cv2.rectangle(frame,(x1,y1),(x2,y2),bbox_color,1)
                (label_width, label_height), baseline = cv2.getTextSize(obj + probability, cv2.FONT_HERSHEY_DUPLEX, 0.4, 1)
                cv2.rectangle(frame,(x1,y1),(x1+label_width+10,y1-label_height-border_offset-10),bbox_color,-1)
                cv2.putText(frame, obj + probability, (x1+5, y1-border_offset-5), cv2.FONT_HERSHEY_DUPLEX, 0.4, (0, 0, 0), 1,
                    cv2.LINE_AA)
            if self.detect_objects == 'human':
                if obj in obj_colors:
                    print(obj)
                    bbox_color = obj_colors[obj]
                    border_offset = 3
                    cv2.rectangle(frame,(x1,y1),(x2,y2),bbox_color,1)
                    (label_width, label_height), baseline = cv2.getTextSize(obj + probability, cv2.FONT_HERSHEY_DUPLEX, 0.4, 1)
                    cv2.rectangle(frame,(x1,y1),(x1+label_width+10,y1-label_height-border_offset-10),bbox_color,-1)
                    cv2.putText(frame, obj + probability, (x1+5, y1-border_offset-5), cv2.FONT_HERSHEY_DUPLEX, 0.4, (0, 0, 0), 1,
                        cv2.LINE_AA)
                    # FACETRACKER
                    if self.face_track == True:
                        cv2.putText(frame,  'Facetrack ON', (0,20), cv2.FONT_HERSHEY_DUPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
                        if obj == 'Human face':
                            cmd = []
                            cv2.rectangle(frame,(int(image_w*0.2),int(image_h*0.2)),(int(image_w*0.8),int(image_h*0.8)),bbox_color,1)
                            try:
                                bbox_center_x = int(x*image_w)
                                bbox_center_y = int(y*image_h)
                                image_center_x = int(image_w/2)
                                image_center_y = int(image_h/2)
                                dist_to_x = bbox_center_x - image_center_x
                                dist_to_y = bbox_center_y - image_center_y
                                bbox_width = x2 - x1
                                cv2.line(frame, (bbox_center_x, bbox_center_y), (image_center_x, image_center_y), (0, 255, 0), thickness=3, lineType=8)
                                cv2.putText(frame,  'DIST_TO_X:' + str(dist_to_x), (0,40), cv2.FONT_HERSHEY_DUPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
                                cv2.putText(frame,  'DIST_TO_Y:'  + str(dist_to_y), (0,60), cv2.FONT_HERSHEY_DUPLEX, 0.4, (255,255, 255), 1, cv2.LINE_AA)
                                if dist_to_x > 0:
                                    if dist_to_x > 200:
                                        cmd.append( 'RIGHT')
                                if dist_to_x < 0:
                                    if dist_to_x < -200:
                                        cmd.append( 'LEFT')
                                if dist_to_y > 0:
                                    if dist_to_y > 100:
                                        cmd.append('DOWN')
                                if dist_to_y < 0:
                                    if dist_to_y < -100:
                                        cmd.append('UP')
                                if bbox_width < (image_w*0.2):
                                    cmd.append('FORWARD')
                                if bbox_width > (image_w*0.3):
                                    cmd.append('BACKWARD')
                            except:
                                print('ERR FACETRACK')
        return frame, cmd

    def on_message(self, message):
        data = json.loads(message)
        frame, cmd = self.highlightImage(data)
        self.frame = frame
        self.cmd = cmd

    def on_error(self, error):
        print(error)


    def on_close(self):
        print("### closed ###")

    def on_open(self):
        print('open')

# Erstellung des Frontends

In [5]:
# Speed of the drone
S = 60

class FrontEnd(object):
    """ Maintains the Tello display and moves it through the keyboard keys.
        Press escape key to quit.
        The controls are:
            - T: Takeoff
            - L: Land
            - Arrow keys: Forward, backward, left and right.
            - A and D: Counter clockwise and clockwise rotations
            - W and S: Up and down.
    """

    def __init__(self):
        # Init pygame
        pygame.init()

        # Creat pygame window
        pygame.display.set_caption("Tello video stream")
        self.screen = pygame.display.set_mode([960, 720])

        # Init Tello object that interacts with the Tello drone
        self.tello = Tello()
        
        # Init ESP Project and receive ESP publisher
        self.esp_project = Esp_project()
        time.sleep(3)
        self.esp_project_sub = Esp_project_sub()
        esp_project_sub_t = threading.Thread(target=self.esp_project_sub.ws.run_forever)
        esp_project_sub_t.daemon = True
        esp_project_sub_t.start()

        # Drone velocities between -100~100
        self.for_back_velocity = 0
        self.left_right_velocity = 0
        self.up_down_velocity = 0
        self.yaw_velocity = 0
        self.speed = 10

        self.send_rc_control = False
        
        self.time_last_frame = 0

        # create update timer
        pygame.time.set_timer(USEREVENT + 1, 50)
    def send_to_esp(self,frame):
        if time.time() - self.time_last_frame > 0.03:
            _, buffer = cv2.imencode('.jpg', frame)
            encoded_string = base64.b64encode(buffer)
            strToSend = 'i, n, ' + str(int(time.time()*100)) + ',' + encoded_string.decode() + ',' + '\n'
            self.esp_project.pub.send(strToSend)
            self.time_last_frame = time.time()

    def run(self):
        if not self.tello.connect():
            print("Tello not connected")
            return
        if not self.tello.set_speed(self.speed):
            print("Not set speed to lowest possible")
            return
        if not self.tello.streamoff():
            print("Could not stop video stream")
            return
        if not self.tello.streamon():
            print("Could not start video stream")
            return

        frame_read = self.tello.get_frame_read()

        should_stop = False
        while not should_stop:
            for event in pygame.event.get():
                if event.type == USEREVENT + 1:
                    self.update()
                elif event.type == QUIT:
                    should_stop = True
                elif event.type == KEYDOWN:
                    if event.key == K_ESCAPE:
                        should_stop = True
                    else:
                        self.keydown(event.key)
                elif event.type == KEYUP:
                    self.keyup(event.key)

            if frame_read.stopped:
                frame_read.stop()
                break

            self.screen.fill([0, 0, 0])
            frame = cv2.cvtColor(frame_read.frame, cv2.COLOR_BGR2RGB)
            self.send_to_esp(frame)
            if self.esp_project_sub.frame is None:
                frame = np.rot90(frame)
                frame = np.flipud(frame)
            if self.esp_project_sub.frame is not None:
                frame = self.esp_project_sub.frame
                frame = np.rot90(frame)
                frame = np.flipud(frame)
                if self.esp_project_sub.face_track == True:
                    cmd = self.esp_project_sub.cmd
                    lr = False
                    td = False
                    fb = False
                    if 'RIGHT' in cmd:
                        self.left_right_velocity = 10
                        lr = True
                    if 'LEFT' in cmd:
                        self.left_right_velocity = -15
                        lr = True
                    if 'UP' in cmd:
                        self.up_down_velocity = 15
                        td = True
                    if 'DOWN' in cmd:
                        self.up_down_velocity = -15
                        td = True
                    if 'FORWARD' in cmd:
                        self.for_back_velocity = 15
                        fb = True
                    if 'BACKWARD' in cmd:
                        self.for_back_velocity = -15
                        fb = True
                    if lr == False:
                        self.left_right_velocity = 0
                    if td == False:
                        self.up_down_velocity = 0
                    if fb == False:
                        self.for_back_velocity = 0

            frame = pygame.surfarray.make_surface(frame)
            self.screen.blit(frame, (0, 0))
            pygame.display.update()
        self.tello.end()

    def keydown(self, key):
        """ Update velocities based on key pressed
        Arguments:
            key: pygame key
        """
        if key == pygame.K_UP:  # set forward velocity
            self.for_back_velocity = S
        elif key == pygame.K_DOWN:  # set backward velocity
            self.for_back_velocity = -S
        elif key == pygame.K_LEFT:  # set left velocity
            self.left_right_velocity = -S
        elif key == pygame.K_RIGHT:  # set right velocity
            self.left_right_velocity = S
        elif key == pygame.K_w:  # set up velocity
            self.up_down_velocity = S
        elif key == pygame.K_s:  # set down velocity
            self.up_down_velocity = -S
        elif key == pygame.K_a:  # set yaw clockwise velocity
            self.yaw_velocity = -S
        elif key == pygame.K_d:  # set yaw counter clockwise velocity
            self.yaw_velocity = S

    def keyup(self, key):
        """ Update velocities based on key released
        Arguments:
            key: pygame key
        """
        if key == pygame.K_UP or key == pygame.K_DOWN:  # set zero forward/backward velocity
            self.for_back_velocity = 0
        elif key == pygame.K_LEFT or key == pygame.K_RIGHT:  # set zero left/right velocity
            self.left_right_velocity = 0
        elif key == pygame.K_w or key == pygame.K_s:  # set zero up/down velocity
            self.up_down_velocity = 0
        elif key == pygame.K_a or key == pygame.K_d:  # set zero yaw velocity
            self.yaw_velocity = 0
        elif key == pygame.K_t:  # takeoff
            self.tello.takeoff()
            self.send_rc_control = True
        elif key == pygame.K_l:  # land
            self.tello.land()
            self.send_rc_control = False
        elif key == pygame.K_f:  # Facetrack ON
            self.esp_project_sub.face_track = True
        elif key == pygame.K_g:  # Facetrack OFF
            self.esp_project_sub.face_track = False
        elif key == pygame.K_h:  # Show human objects
            self.esp_project_sub.detect_objects = 'human'
        elif key == pygame.K_j:  # Show all objects
            self.esp_project_sub.detect_objects = 'all'

    def update(self):
        """ Update routine. Send velocities to Tello."""
        if self.send_rc_control:
            self.tello.send_rc_control(self.left_right_velocity, self.for_back_velocity, self.up_down_velocity,
                                       self.yaw_velocity)

In [ ]:
frontend = FrontEnd()
frontend.run()

ESP Project loaded


Send command: command
INFO:djitellopy:Send command: command
Response: ok
INFO:djitellopy:Response: ok
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: Expecting value: line 1 column 1 (char 0)
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: 'events'
Send command: speed 10
INFO:djitellopy:Send command: speed 10
Response: ok
INFO:djitellopy:Response: ok
Send command: streamoff
INFO:djitellopy:Send command: streamoff
Response: ok
INFO:djitellopy:Response: ok
Send command: streamon
INFO:djitellopy:Send command: streamon
Response: ok
INFO:djitellopy:Response: ok


open


ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: 'Esp_project_sub' object has no attribute 'detect_objects'
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: 'Esp_project_sub' object has no attribute 'detect_objects'
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: 'Esp_project_sub' object has no attribute 'detect_objects'
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: 'Esp_project_sub' object has no attribute 'detect_objects'
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: 'Esp_project_sub' object has no attribute 'detect_objects'
ERROR:websocket:error from callback

Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
ERROR:websocket:error from callback <bound method Esp_project_sub.on_message of <__main__.Esp_project_sub object at 0x7ff110c09d90>>: '_Object20_'
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): rc 0 0 0 0
Send command (no expect response): rc 0 0 0 0
INFO:djitellopy:Send command (no expect response): 

...

Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0


Person
Person
Human face
Human face
Human face
Person
Person
Person
Person
Person
Person
Person
Human face
Human face
Human face
Person
Person
Person
Person
Person
Person
Person
Human face
Human face
Person
Person
Person
Person
Person
Person
Human face
Human face
Person
Person
Person
Person
Person


Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0


Person
Person
Human face
Human face
Person
Person
Person
Person
Person
Person
Person
Person
Human face
Human face
Human face
Person
Person
Person
Person
Person
Person
Person
Human face
Human face
Person
Person
Person
Person
Person


Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
Send command (no expect response): rc 10 15 15 0
INFO:djitellopy:Send command (no expect response): rc 10 15 15 0
